<a href="https://colab.research.google.com/github/James1996GitHub/Age-Classifier/blob/main/Code_Dump.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Mounts the Gdive to the script. This allows us to acccess the files from our Google drive account (import Test/Train data).

In [ ]:

from google.colab import drive
drive.mount('/content/gdrive')

A linst of all our imports that we will bw using in this project.


In [ ]:
import os 
import pandas as pd
import csv
import numpy
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import PIL.Image
import cv2


## Declaring training and testing data as variables for future convienience. 

In [ ]:
TrainDir = '/content/gdrive/MyDrive/train_DETg9GD.zip (Unzipped Files)/Train'
TestDir = '/content/gdrive/MyDrive/Actors Age Project /Test/test.zip (Unzipped Files)/Test'
NumTrainingData = len(os.listdir(TrainDir))
NumTestingData = len(os.listdir(TestDir))
print('Total training images:',NumTrainingData) #returns 3 more images than expected.
print('Total testing images:',NumTestingData)
print('Total images:',NumTestingData + NumTrainingData)

Total training images: 19909
Total testing images: 6636
Total images: 26545


Building an idea of what the shape of our data pipline will look like.

In [ ]:
TrainCSV = '/content/gdrive/MyDrive/train_DETg9GD.zip (Unzipped Files)/train.csv'
TestCSV = '/content/gdrive/MyDrive/Actors Age Project /Test/test.zip (Unzipped Files)/test.csv'
CSVLables = ['Image File', 'Age Group']
TrainDataShape = pd.read_csv(TrainCSV, names=CSVLables, skiprows=[0])
TestDataShape = pd.read_csv(TestCSV, names=CSVLables, skiprows=[0])
print('Training datashape:',TrainDataShape.shape)
print('Testing datashape:',TestDataShape.shape)
print('Total rows:',TestDataShape.shape[0]+TrainDataShape.shape[0])
TrainDataShape.head()

Training datashape: (19906, 2)
Testing datashape: (6636, 2)
Total rows: 26542


,Image File,Age Group
0,377.jpg,MIDDLE
1,17814.jpg,YOUNG
2,21283.jpg,MIDDLE
3,16496.jpg,YOUNG
4,4487.jpg,MIDDLE


In [ ]:
TestDataShape

,Image File,Age Group
0,25321.jpg,NaN
1,989.jpg,NaN
2,19277.jpg,NaN
3,13093.jpg,NaN
4,5367.jpg,NaN
...,...,...
6631,1876.jpg,NaN
6632,14940.jpg,NaN
6633,3638.jpg,NaN
6634,376.jpg,NaN


DATA AUGMENTATION WOULD BE DONE IN THIS STEP IN THIS STAGE using the Keras ImageDataGenerator method/function.

Epoch size = Total Train Data /batch size

For 19,909 = num training 

A good batch size = 43

epoch = 1607 / 57 = 463

For 19906 

batch size = 74

epoch = 269

In [ ]:
batch_size = 43
epochs = 20
Image_size = [40,40] # See in build the model section: old value = [23,23]

Data Preprocessing

In [ ]:
train_rescale=ImageDataGenerator(rescale=1./255.,validation_split=0.3)
test_rescale=ImageDataGenerator(rescale=1./255.)

#training generator
train_generator=train_rescale.flow_from_dataframe(
    dataframe=TrainDataShape,
    directory=TrainDir, 
    x_col=CSVLables[0],
    y_col=CSVLables[1],
    batch_size=32,
    shuffle=True,
    subset = 'training',
    class_mode="categorical",
    target_size= Image_size)

#validation generator
validation_generator=train_rescale.flow_from_dataframe(
    dataframe=TrainDataShape,
    directory=TrainDir,
    x_col=CSVLables[0],
    y_col=CSVLables[1],
    batch_size=32,
    shuffle=True,
    subset = 'validation',
    class_mode="categorical",
    target_size= Image_size)

#test generator
test_generator=test_rescale.flow_from_dataframe(
    dataframe=TestDataShape,
    directory=TestDir,
    x_col=CSVLables[0],
    y_col=None,
    batch_size=32,
    class_mode=None,
    target_size= Image_size)

Found 13935 validated image filenames belonging to 3 classes.
Found 5971 validated image filenames belonging to 3 classes.
Found 6636 validated image filenames.


Next we are building a Alexnet CNN archetecture.
Build the model https://www.tensorflow.org/tutorials/images/classification

In [ ]:
num_classes = 3
img_height =  40  
img_width =  40 

model = Sequential([
  layers.experimental.preprocessing.Rescaling(1./255, input_shape=(img_height, img_width, 3)),  
  layers.Conv2D(96,kernel_size=(11,11), padding='same', activation='relu', input_shape=(img_height, img_width, 3)),
  layers.MaxPooling2D(),
  layers.BatchNormalization(),
  layers.Conv2D(256,kernel_size=(5,5), padding='same', activation='relu'),
  layers.BatchNormalization(),
  layers.MaxPooling2D(),
  layers.Conv2D(384,kernel_size=(3,3), padding='same', activation='relu'),
  layers.BatchNormalization(),
  layers.Conv2D(384,kernel_size=(3,3), padding='same', activation='relu'),
  layers.BatchNormalization(),
  layers.Conv2D(256,kernel_size=(3,3), padding='same', activation='relu'),
  layers.BatchNormalization(),
  layers.MaxPooling2D(),
  layers.Flatten(),
  layers.Dense(84, activation='relu'),
  layers.Dropout(0.3),
  layers.Dense(4096, activation='relu'),
  layers.Dropout(0.2),
  layers.Dense(num_classes, activation='softmax')
])
model.compile(
    loss='categorical_crossentropy', 
    metrics=[tf.keras.metrics.Precision()],
    optimizer = 'sgd'
)

Train the model - at the moment this is using the incorrect batch size

In [ ]:

STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=validation_generator.n//validation_generator.batch_size
STEP_SIZE_TEST=test_generator.n//test_generator.batch_size

salisburyNet = model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=validation_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=epochs
)


/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/20
435/435 [==============================] - 4751s 11s/step - loss: 0.8629 - precision: 0.6340 - val_loss: 1.1974 - val_precision: 0.0000e+00
Epoch 2/20
435/435 [==============================] - 1225s 3s/step - loss: 0.7572 - precision: 0.7001 - val_loss: 0.8544 - val_precision: 0.6258
Epoch 3/20
435/435 [==============================] - 1219s 3s/step - loss: 0.7025 - precision: 0.7298 - val_loss: 0.7488 - val_precision: 0.6734
Epoch 4/20
435/435 [==============================] - 1212s 3s/step - loss: 0.6522 - precision: 0.7506 - val_loss: 0.6670 - val_precision: 0.7396
Epoch 5/20
435/435 [==============================] - 1213s 3s/step - loss: 0.6083 - precision: 0.7675 - val_loss: 0.6969 - val_precision: 0.7174
Epoch 6/20
435/435 [==============================] - 1215s 3s/step - loss: 0.5630 - precision: 0.7915 - val_loss: 0.6324 - val_precision: 0.7382
Epoch 7/20
435/435 [==============================] - 1226s 3s/step - loss: 0.5178 - precision: 0.8091 - val_loss: 0.66

In [ ]:
model.save('/content/gdrive/MyDrive/Actors Age Project.h')

INFO:tensorflow:Assets written to: /content/gdrive/MyDrive/Actors Age Project/assets


In [ ]:
salisburyNet = tf.keras.models.load_model('/content/gdrive/MyDrive/Actors Age Project.h')